<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/TopicosAvanzadosComputacion2/blob/master/DL_L11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos Avanzados de Computación II
## Dr. Carlos Villaseñor
## Lección 11 - Modelos Secuenciales Parte I
### Practica de generación de nombres con LSTM


Primeramente importamos las paqueterias necesarias

In [2]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from google.colab import files

Necesitaremos una base de datos de nombres para poder aprender a generarlos

In [3]:
uploaded = files.upload()

Saving names.txt to names.txt


Vamos a abrir el archivo con conmbres y limpar un poco los datos

In [4]:
text = open('names.txt').read()
text = text.lower()
names = text.split('\n')
names = list(filter(lambda name: ' ' not in name, names))
names = list(filter(lambda name: '-' not in name, names))
names = list(filter(lambda name: "'" not in name, names))
names = [name + '.' for name in names]
names = np.random.permutation(names)
print(names)

['gunther.' 'faydra.' 'taryn.' ... 'hammad.' 'avrit.' 'pryce.']


Vamos a crear un diccionario que con el cual podamos mapear de caracteres a números

In [5]:
char_to_index = dict( (chr(i+96), i) for i in range(1,27))
char_to_index[' '] = 0
char_to_index['.'] = 27
print(char_to_index)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, ' ': 0, '.': 27}


También necesitaremos el mapeo inverso

In [6]:
index_to_char = dict( (i, chr(i+96)) for i in range(1,27))
index_to_char[0] = ' '
index_to_char[27] = '.'
print(index_to_char)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: ' ', 27: '.'}


Calculemos el número de nombres y el tamaño máximo que un nombre puede tener

In [7]:
num_names = len(names)
max_name_size = len(max(names, key=len))
num_chars = len(char_to_index)
print(max_name_size)

14


Vamos a códificar todos los nombres y a crear un dataset con los nombres y su retardo

In [8]:
x = np.zeros((num_names, max_name_size, num_chars))
y = np.zeros((num_names, max_name_size, num_chars))
print(x.shape)

for i, name in enumerate(names):
  for j, c in enumerate(name):
    x[i, j, char_to_index[name[j]]] = 1
    if j < len(name)-1:
      y[i, j, char_to_index[name[j+1]]] = 1

(7904, 14, 28)


Intanciamos nuestra red recurrente LSTM

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_name_size, num_chars),return_sequences=True))
model.add(Dropout(0.25))
model.add(Dense(num_chars, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 14, 128)           80384     
_________________________________________________________________
dropout (Dropout)            (None, 14, 128)           0         
_________________________________________________________________
dense (Dense)                (None, 14, 28)            3612      
Total params: 83,996
Trainable params: 83,996
Non-trainable params: 0
_________________________________________________________________


Momento de entrenar!!

In [10]:
model.fit(x, y, batch_size=128, epochs=100)

Epoch 1/100
62/62 [==============================] - 2s 38ms/step - loss: 1.2576
Epoch 2/100
62/62 [==============================] - 2s 38ms/step - loss: 1.1328
Epoch 3/100
62/62 [==============================] - 2s 38ms/step - loss: 1.0877
Epoch 4/100
62/62 [==============================] - 2s 37ms/step - loss: 1.0372
Epoch 5/100
62/62 [==============================] - 2s 38ms/step - loss: 0.9917
Epoch 6/100
62/62 [==============================] - 2s 38ms/step - loss: 0.9601
Epoch 7/100
62/62 [==============================] - 2s 37ms/step - loss: 0.9372
Epoch 8/100
62/62 [==============================] - 2s 37ms/step - loss: 0.9226
Epoch 9/100
62/62 [==============================] - 2s 38ms/step - loss: 0.9085
Epoch 10/100
62/62 [==============================] - 2s 38ms/step - loss: 0.8982
Epoch 11/100
62/62 [==============================] - 2s 38ms/step - loss: 0.8884
Epoch 12/100
62/62 [==============================] - 2s 38ms/step - loss: 0.8806
Epoch 13/100
62/62 [=====

El modelo está entrenado, pero ¿Cómo generamos nombres?!!

Para esto está el código siguiente!!


In [24]:
# Correr un ejemplo
name = ''
x = np.zeros((1,max_name_size, num_chars))
first = True
end = False
i = 0
while end==False:
  if first == True:
    probs = list(model.predict(x)[0,i])[1:-1]
    probs = probs/ np.sum(probs)
    index = np.random.choice(range(1,num_chars-1),p=probs)
    first = False
  else:
    probs = list(model.predict(x)[0,i])
    probs = probs/ np.sum(probs)
    index = np.random.choice(range(num_chars),p=probs)
  if i == max_name_size-1:
    break
    character = '.'
    end = True
  else:
    character = index_to_char[index]
  name += character
  x[0, i+1, index] = 1
  i += 1
  if character == '.':
    end = True

print(name.capitalize())
print('Memorized: ', name in names)

Edtre.
Memorized:  False
